In [ ]:
using InteractiveUtils
versioninfo()

In [ ]:
using Pkg
Pkg.add(["CSV", "DataFrames", "MLJ", "MLJFlux", "Flux", "Plots", "StatsBase", "CategoricalArrays", "MLJBase","Glob"])
Pkg.add(["DecisionTree","EvoTrees","MLJ","MLJFlux","Flux","StatsBase","MLJDecisionTreeInterface","NearestNeighborModels","MLJLinearModels","LIBSVM","EvoTrees","LightGBM","MLJLIBSVMInterface","MLJLightGBMInterface","MLJDecisionTreeInterface","MLJLinearModels","MLJFlux","MLJNeuralNetworks","MLJModels","MLJBase","MLJModelInterface","MLJModelInterfaceUtils","Glob"])

In [ ]:
using CSV,DataFrames, MLJ, Flux,Plots,StatsBase,CategoricalArrays,MLJFlux

In [ ]:
import Glob
path = "./cleaneddata"
files = Glob.glob("S*_cleaned_domain_features.csv", path)
df_all = DataFrame[]
for file in files
    push!(df_all,CSV.read(file,DataFrame))
end 

data = reduce(vcat,df_all)
first(data,5)

In [ ]:
if "sid" in names(data)
    select!(data, Not(:sid))
end

foreach(col -> replace!(col, missing => 0.0), eachcol(data))

valid_stages = ["W","N","R"]
data = filter(row -> row.Sleep_Stage in valid_stages, data)
label_mapping = Dict("W"=> "Wake", "N" => "NREM", "R" => "REM")
data.Sleep_Stage = CategoricalArray([label_mapping[String(x)] for x in data.Sleep_Stage])

y = data.Sleep_Stage
X = select(data, Not(:Sleep_Stage))

In [ ]:
using MLJ, MLJFlux, Flux, StatsBase, DataFrames, CategoricalArrays
using DecisionTree, NearestNeighborModels, MLJLinearModels, LIBSVM, EvoTrees

@load DecisionTreeClassifier pkg=DecisionTree
@load KNNClassifier pkg=NearestNeighborModels
@load LogisticClassifier pkg=MLJLinearModels
@load SVC pkg=LIBSVM
@load EvoTreeClassifier pkg=EvoTrees

input_dim = size(X, 2)
output_dim = length(levels(y))
deep_net = Chain(
    Dense(input_dim, 128, relu),
    Dense(128, 64, relu),
    Dense(64, output_dim),
    Flux.softmax
)
deep_builder = MLJFlux.GenericBuilder(deep_net)

model_list = [
    Standardizer() |> DecisionTreeClassifier(max_depth=3),
    Standardizer() |> KNNClassifier(K=5),
    Standardizer() |> LogisticClassifier(),
    Standardizer() |> SVC(),
    Standardizer() |> EvoTreeClassifier(nrounds=100),
    Standardizer() |> NeuralNetworkClassifier(
        builder=deep_builder, 
        loss=Flux.Losses.crossentropy, 
        optimiser=Flux.ADAM(), 
        epochs=20
    )
]

cv = CV(nfolds=3, shuffle=true)

for model in model_list
    println("\n▶ Evaluating: ", model)
    try
        mach = machine(model, X, y)
        res = evaluate!(mach, resampling=cv,
            measures=[accuracy, macro_f1score, micro_f1score, cross_entropy],
            verbosity=0)
        acc = round(mean(res.measurement[1]), digits=4)
        println("→ Accuracy: ", acc)

        MLJ.fit!(mach)
        model_name = string(typeof(model).name.name)
        filename = model_name * ".jls"
        MLJ.save(filename, mach)
        println("Saved model to $filename")

        ŷ = predict_mode(mach, X)
        cm = confusion_matrix(y, ŷ)
        println("Confusion Matrix:\n", cm)
    catch err
        println("Error with model $(model): ", err)
    end
end

In [ ]:
import Pkg; Pkg.add("DecisionTree")
import Pkg; Pkg.add("NearestNeighborModels")
import Pkg; Pkg.add("MLJLinearModels")
import Pkg; Pkg.add("LIBSVM")
import Pkg; Pkg.add("EvoTrees")
import Pkg; Pkg.add("MLJDecisionTreeInterface")